In [ ]:
import os, random
import numpy as np
import tensorflow as tf
import json

def load_json(filename):
    with open(filename) as json_file:
        data = json.load(json_file)
        return data

def show_files(label_dir):
    all_files = []
    file_list = os.listdir(label_dir)
    for file in file_list:
        cur_path = os.path.join(label_dir, file)
        root, extension = os.path.splitext(cur_path)
        ext_lower = extension.lower()
        if ext_lower == ".json":
            all_files.append(cur_path)
    return all_files

def getX(points):
    size = len(points)
    if size < 6 : # 过滤误触
        return None
    sample_count = 9
    tensor_size = 6
    step = size * 1.0 / sample_count
    x = []
    for i in range(sample_count):
        p = points[int(i*step)]
        if len(p) == 6:
            x.append(p) # x, y, w, h, density, dtime
    return x

def getY(label):
    if label == "right":
        return 1
    return 0

def obtainXY(data_dir, shuffle=False):
    X = []
    Y = []
    classes =  ["left", "right"]
    all_file_count = 0
    for label in classes:
        all_files = show_files(data_dir + str(label))
        all_file_count += len(all_files)
        for file in all_files:
            one_tracker = load_json(file)
            for points in one_tracker:
                x = getX(points)
                y = getY(label)
                if x is not None:
                    X.append(x)
                    Y.append(y)
    print("all file count=", all_file_count)
    X = np.array(X)
    Y = np.array(Y)
    if shuffle:
        indexs = [i for i in range(len(X))] 
        random.shuffle(indexs) #乱序
        X = X[indexs]
        Y = Y[indexs]
    return X, Y

dataset_dir = "./dataset/"
X, Y = obtainXY(dataset_dir, shuffle=True)
print("train data:", X.shape, Y.shape)

train_x = X
train_y = Y
# input output shape
print("tran data shape:", train_x.shape, train_y.shape)

dataset_test_dir = "./dataset-test/"
vX, vY = obtainXY(dataset_test_dir)
print("test data:", vX.shape, vY.shape)

In [ ]:
from collections import Counter

def print_pecent(data, class_num=2):
    y_ = list(data)
    size = len(y_)
    class_counter = Counter(y_)
    print(class_counter, size)
    for i in range(class_num):
        print(i, "占比:", round(class_counter[i]/size, 3))

print_pecent(Y)

In [ ]:
from keras import layers, regularizers
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

# 卷积神经网络
def creat_cnn(input_shape=(9, 6), print_summary=False):
    model = Sequential()
    model.add(layers.Conv1D(6, kernel_size=3, input_shape=input_shape, padding='same', activation='relu'))
    model.add(layers.Conv1D(12, kernel_size=3, input_shape=input_shape, padding='same', activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=3, strides=None, padding='same', data_format='channels_last'))
    model.add(layers.Conv1D(24, kernel_size=3, input_shape=input_shape, padding='same', activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    if print_summary:
        model.summary()
    return model

# 训练模型
def train_net(train_x, train_y, batch_size, epochs, load_save_model=False):
    tflite_name = "mymodel.tflite"
    model_path = './model/'
    if load_save_model:
        print('train load save model path', model_path)
        model = load_model(model_path)
    else:
        print('train create new model')
        model = creat_cnn(print_summary=True) # cnn卷积神经网络
    model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1)
    model.save(model_path)
    
    #save tflite model
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    open(tflite_name, "wb").write(tflite_model)
    return model

# 验证模型
def verify_model(model, valid_x, valid_y, num_classes=2):
    y_pred = model.predict(valid_x)
    y_pred = np.round(y_pred).astype(int)
    size = [0 for i in range(num_classes)]
    correct = [0 for i in range(num_classes)]
    for i in range(len(y_pred)):
        label = valid_y[i]
        pred = y_pred[i]
        size[label] += 1
        if pred == label:
            correct[label] += 1
    print("total accuracy:", sum(correct)/sum(size))
    for i in range(num_classes):
        print("label:", str(i), " accuracy:", correct[i]/max(size[i], 1))

In [ ]:
# 训练卷积网络模型
model = train_net(train_x, train_y, batch_size=16, epochs=24, load_save_model=False)
# 验证模型
verify_model(model, vX, vY)

In [ ]:
# retrain model
#train_net(train_x, train_y, batch_size=16, epochs=8, load_save_model=True)